## Traditional Similarity search methods

1. Jaccard = intersection(A,B) / union(A,B). For example, A = {1,2,3} and B = {2,4}, our resulkting Jaccard = 0.25
2. Levenshtein (similar to Jaccard but requires preprocessign for n-gram representation)

In [4]:
def jaccard(sentc_one: str, sentc_two: str):
    # Convert to sets
    sentc_one = set(sentc_one.split())
    sentc_two = set(sentc_two.split())
    # Calculate
    shared = sentc_one.intersection(sentc_two)
    union = sentc_one.union(sentc_two)
    return len(shared) / len(union)





In [12]:
jaccard('Paul is cool', 'Cool person forever Paul is coolest is coolest')

0.2857142857142857

## Sparse and Dense Representations

1. TF-IDF (Sparse)

Term Frequency = f(q, D) / f(t, D)

f(q, D) - Frequency of query q in Document D
f(t, D) - Frequency of all terms in Document D (total num terms)

Inverse Document Frequency or IDF = log (N / N(q='forest'))
It is calculate for each document

N is the total number of documents
N(q="forest") is number of documents containing the query forest

2. BM25 (Sparse)
3. SBERT (Dense)

In [18]:
a = "purple is the best city in the forest".split()
b = "there is an art to getting your way and throwing bananas on to the street is not it".split()
c = "it is not often you find soggy bananas on the street".split()


In [14]:
a

['purple', 'is', 'the', 'best', 'city', 'in', 'the', 'forest']

In [20]:
import numpy as np

# we'll merge all docs into a list of lists for easier calculation of IDF
docs = [a , b, c]
# print(docs)

def tfidf(word, sentence):
    # term frequency
    freq = sentence.count(word)
    tf = freq / len(sentence)
    
    # inverse document frequency
    idf = np.log10(len(docs) / sum([1 for doc in docs if word in doc]))

    # tf-idf
    tf_idf = round(tf*idf, 4)

    return tf_idf


In [22]:
tfidf('forest', a)

0.0596

In [23]:
# Create Document vectors
vocab = set(a+b+c)
vocab

{'an',
 'and',
 'art',
 'bananas',
 'best',
 'city',
 'find',
 'forest',
 'getting',
 'in',
 'is',
 'it',
 'not',
 'often',
 'on',
 'purple',
 'soggy',
 'street',
 'the',
 'there',
 'throwing',
 'to',
 'way',
 'you',
 'your'}

In [25]:
# Mirror our vocab as vector, for each document

vec_a = []
vec_b = []
vec_c = []

for word in vocab:
    vec_a.append(tfidf(word, a))
    vec_b.append(tfidf(word, b))
    vec_c.append(tfidf(word, c))


In [26]:
vec_b

[0.0,
 0.0265,
 0.0265,
 0.053,
 0.0098,
 0.0,
 0.0265,
 0.0,
 0.0265,
 0.0,
 0.0265,
 0.0098,
 0.0,
 0.0,
 0.0098,
 0.0,
 0.0098,
 0.0,
 0.0098,
 0.0265,
 0.0,
 0.0,
 0.0,
 0.0265,
 0.0265]

## BM25
BM25 is an optimized version of TFIDF

One of the problems with the TFIDF method is that as the frequency of queries found in a document increase, the score increases linearly.

Imagine 1000 word article with word dog appearing 10 times = good chance article is talking about dog

Now if you double words that mention dog to 20, TFIDF will double as well. 

Logically that shouldnt be the case. That means that the original document with dog 10times is a document that is half as relevant to dogs as the article with dog 20 times. BM25 tries to normalize this exaggeration. 

In [3]:
a = "purple is the best city in the forest".split()
b = "there is an art to getting your way and throwing bananas on to the street is not it".split()
c = "it is not often you find soggy bananas on the street".split()
d = "green should have smelled more tranquil but somehow it just tasted rotten".split()
e = "joyce enjoyed eating pancakes with ketchup".split()
f = "as the asteroid hurtled toward earth becky was upset her dentist appointment had been canceled".split()

In [5]:
import numpy as np

docs = [a, b, c, d, e, f]
N = len(docs) # Number of documents

# Recall tfidf
def tfidf(word, sentence):
    # term frequency
    freq = sentence.count(word)
    tf = freq / len(sentence)
    
    # inverse document frequency
    N_q = sum([1 for doc in docs if word in doc]) # Number of docs that contain the word  
    idf = np.log10(N / N_q)

    # tf-idf
    tf_idf = round(tf*idf, 4)

    return tf_idf



In [6]:
# Optimization new features
# 1. Average Document length (Sum of All len(sent) across docs / N)
avgdl = sum(len(sentence) for sentence in docs) / N 

# 2. k and b - Adjustable parameters that control ...


In [7]:
N = len(docs) # Number of documents
#bm25 implementation
def bm25(word, sentence, k=1.2, b=0.75):
    # term frequency
    freq = sentence.count(word) 
    
    tf = (freq * (k + 1)) / (freq + k * (1 - b + b * len(sentence) / avgdl))

    # inverse document frequency
    N_q = sum([1 for doc in docs if word in doc]) # Number of docs that contain the word    
    idf = np.log(((N - N_q + 0.5) / (N_q + 0.5)) + 1)

    return round(tf*idf, 4)
    

In [9]:
bm25('purple', a)

1.7677

### Comparing TF-IDF and BM25 algorithms
The TF-IDF score increases linearly with the frequency for the matching term 

The BM25 algorithm increases logarithmically.

In most cases BM25 is more realistic.

## Dense vectors with SBERT
The effect of using dense vectors is that it lets you represent words in a more meaningful manner

For example words like hi == hello in their vector similarity
